# NBA Predictive Analysis

This notebook uses historical spread coverage analysis to identify today's games with high probability betting opportunities.

## Step 1: Get Today's Games

Fetch today's games from the Odds API with proper EST timezone handling.

In [1]:
import pandas as pd
import sys
import os
from pathlib import Path
from datetime import datetime, timedelta, timezone
import time

# Add src directory to path
project_root = Path().resolve().parent
sys.path.insert(0, str(project_root / 'src'))

from odds_api_client import OddsAPIClient, OddsAPIError
import config

print("Libraries imported successfully")

# Helper function for timezone parsing (used in multiple cells)
def parse_commence_time_to_est(commence_time):
    """Parse commence_time from API and convert to EST datetime"""
    try:
        event_time_utc = pd.to_datetime(commence_time)
        if event_time_utc.tzinfo is None:
            event_time_utc = event_time_utc.replace(tzinfo=timezone.utc)
        elif isinstance(event_time_utc, pd.Timestamp):
            if event_time_utc.tz is None:
                event_time_utc = event_time_utc.tz_localize('UTC').to_pydatetime()
            else:
                event_time_utc = event_time_utc.to_pydatetime()
        
        # Convert to EST
        est = timezone(timedelta(hours=-5))
        event_time_est = event_time_utc.astimezone(est)
        return event_time_est
    except Exception as e:
        return None

Libraries imported successfully


In [2]:
# Initialize API client
client = OddsAPIClient()
api_sport_key = config.get_sport_api_key('nba')

# Get today's date in EST (matching our data collection pattern)
est = timezone(timedelta(hours=-5))
today_est = datetime.now(est).date()

print(f"Today's date (EST): {today_est}")
print(f"Looking for games scheduled for today...")

# Get upcoming games with odds
try:
    time.sleep(config.API_RATE_LIMIT_DELAY)
    endpoint = f"sports/{api_sport_key}/odds"
    params = {
        "regions": "us",
        "markets": "spreads",
        "oddsFormat": "american",
        "dateFormat": "iso"
    }
    
    upcoming_odds = client._make_request(endpoint, params)
    
    print(f"✓ Fetched data from API")
    print(f"Type: {type(upcoming_odds)}")
    
    # Filter to only games happening today (using EST timezone)
    if isinstance(upcoming_odds, list):
        today_games = []
        for game in upcoming_odds:
            if not isinstance(game, dict):
                continue
            
            commence_time = game.get('commence_time')
            if commence_time:
                event_time_est = parse_commence_time_to_est(commence_time)
                if event_time_est and event_time_est.date() == today_est:
                    today_games.append(game)
        
        upcoming_odds = today_games
        print(f"Found {len(upcoming_odds)} games scheduled for today (EST)")
    elif isinstance(upcoming_odds, dict):
        print(f"Response keys: {list(upcoming_odds.keys())}")
        upcoming_odds = []
    
except Exception as e:
    print(f"Error: {e}")
    import traceback
    traceback.print_exc()
    upcoming_odds = []

Today's date (EST): 2025-12-12
Looking for games scheduled for today...
✓ Fetched data from API
Type: <class 'list'>
Found 7 games scheduled for today (EST)


In [3]:
# Process and display today's games in a clean DataFrame
if upcoming_odds and len(upcoming_odds) > 0:
    games_data = []
    
    for game in upcoming_odds:
        if not isinstance(game, dict):
            continue
        
        # Extract basic game info
        event_id = game.get('id', '')
        home_team = game.get('home_team', '')
        away_team = game.get('away_team', '')
        commence_time = game.get('commence_time', '')
        
        # Parse commence_time using helper function
        event_time_est = parse_commence_time_to_est(commence_time)
        game_time_str = event_time_est.strftime('%Y-%m-%d %H:%M:%S %Z') if event_time_est else None
        
        # Extract current spread from DraftKings
        current_spread = None
        spread_odds = None
        bookmakers = game.get('bookmakers', [])
        
        for bookmaker in bookmakers:
            if 'draftkings' in bookmaker.get('key', '').lower():
                for market in bookmaker.get('markets', []):
                    if market.get('key') == 'spreads':
                        outcomes = market.get('outcomes', [])
                        if len(outcomes) >= 2:
                            # Find home team outcome
                            for outcome in outcomes:
                                outcome_name = outcome.get('name', '')
                                if home_team.lower() in outcome_name.lower() or outcome_name.lower() in home_team.lower():
                                    current_spread = outcome.get('point')
                                    spread_odds = outcome.get('price')
                                    break
                            
                            # If not found, use first outcome
                            if current_spread is None:
                                current_spread = outcomes[0].get('point')
                                spread_odds = outcomes[0].get('price')
                        break
                break
        
        games_data.append({
            'event_id': event_id,
            'game_time_est': game_time_str,
            'home_team': home_team,
            'away_team': away_team,
            'current_spread': current_spread,
            'spread_odds': spread_odds
        })
    
    # Create DataFrame
    df_today = pd.DataFrame(games_data)
    
    if len(df_today) > 0:
        print(f"\nToday's NBA Games ({len(df_today)} games):")
        print("="*80)
        print(df_today.to_string(index=False))
    else:
        print("No games data to display")
else:
    print("No games scheduled for today")
    df_today = pd.DataFrame()


Today's NBA Games (7 games):
                        event_id                 game_time_est             home_team              away_team  current_spread  spread_odds
ed94a98c5965c7f5edaf8e956db10d63 2025-12-12 19:10:00 UTC-05:00       Detroit Pistons          Atlanta Hawks            -7.5         -110
1718bd83d0e967f099a883a4df92d7a5 2025-12-12 19:10:00 UTC-05:00     Charlotte Hornets          Chicago Bulls             2.5         -102
a365ef6ee881da6cbcbb41b19a203942 2025-12-12 19:10:00 UTC-05:00    Washington Wizards    Cleveland Cavaliers            14.5         -105
3b851b37fe204032efa750f6e6cde66e 2025-12-12 19:10:00 UTC-05:00    Philadelphia 76ers         Indiana Pacers            -6.5         -120
0bc4679b4f3eed7ea44f40f68984214c 2025-12-12 20:10:00 UTC-05:00     Memphis Grizzlies              Utah Jazz            -7.5         -105
f43f63c4089813a0e3aa0ec0c9ccdd1e 2025-12-12 20:40:00 UTC-05:00      Dallas Mavericks          Brooklyn Nets            -7.5         -120
3d88af9ad2a

## Step 2: Load Historical Team Performance Data

Load the historical NBA data to see how each team has performed against the spread this season.

In [4]:
# Load historical NBA season results
historical_file = project_root / 'data' / 'results' / 'nba_season_results.xlsx'

if historical_file.exists():
    df_historical = pd.read_excel(historical_file)
    print(f"✓ Loaded {len(df_historical)} historical games")
    print(f"Date range: {df_historical['game_date'].min()} to {df_historical['game_date'].max()}")
    print(f"\nFirst few rows:")
    print(df_historical.head())
else:
    print(f"✗ Historical data file not found at {historical_file}")
    df_historical = pd.DataFrame()

✓ Loaded 366 historical games
Date range: 2025-10-21 00:00:00 to 2025-12-11 00:00:00

First few rows:
   game_date              home_team              away_team  closing_spread  \
0 2025-10-21  Oklahoma City Thunder        Houston Rockets            -6.5   
1 2025-10-21     Los Angeles Lakers  Golden State Warriors             2.5   
2 2025-10-22          Orlando Magic             Miami Heat            -8.5   
3 2025-10-22          Atlanta Hawks        Toronto Raptors            -5.5   
4 2025-10-22          Chicago Bulls        Detroit Pistons             3.5   

   home_score  away_score  spread_result_difference  
0         125         124                      -5.5  
1         109         119                      -7.5  
2         125         121                      -4.5  
3         118         138                     -25.5  
4         115         111                       7.5  


## Step 3: Calculate Team Spread Coverage Statistics

For each team, calculate:
- How often they cover the spread (cover %)
- How many games they've played
- Their average spread_result_difference

In [5]:
# Calculate team spread coverage statistics
if len(df_historical) > 0:
    # Filter to only completed games with scores and spreads
    df_completed = df_historical[
        (df_historical['home_score'].notna()) & 
        (df_historical['away_score'].notna()) & 
        (df_historical['closing_spread'].notna()) &
        (df_historical['spread_result_difference'].notna())
    ].copy()
    
    print(f"Completed games with all data: {len(df_completed)}")
    
    # Calculate stats for each team (both as home and away)
    team_stats = []
    all_teams = set(df_completed['home_team'].unique()) | set(df_completed['away_team'].unique())
    
    for team in all_teams:
        # Games where team was home
        home_games = df_completed[df_completed['home_team'] == team].copy()
        # Games where team was away
        away_games = df_completed[df_completed['away_team'] == team].copy()
        
        # For home games: positive spread_result_difference = cover
        # For away games: negative spread_result_difference = cover
        home_covers = (home_games['spread_result_difference'] > 0).sum() if len(home_games) > 0 else 0
        away_covers = (away_games['spread_result_difference'] < 0).sum() if len(away_games) > 0 else 0
        
        total_games = len(home_games) + len(away_games)
        total_covers = home_covers + away_covers
        
        if total_games > 0:
            cover_pct = (total_covers / total_games) * 100
            avg_spread_diff = (
                (home_games['spread_result_difference'].sum() if len(home_games) > 0 else 0) +
                (away_games['spread_result_difference'].sum() * -1 if len(away_games) > 0 else 0)
            ) / total_games
            
            team_stats.append({
                'team': team,
                'total_games': total_games,
                'covers': total_covers,
                'non_covers': total_games - total_covers,
                'cover_pct': cover_pct,
                'avg_spread_diff': avg_spread_diff,
                'home_games': len(home_games),
                'away_games': len(away_games)
            })
    
    # Create DataFrame of team stats
    df_team_stats = pd.DataFrame(team_stats)
    df_team_stats = df_team_stats.sort_values('cover_pct', ascending=False)
    
    print(f"\n✓ Calculated stats for {len(df_team_stats)} teams")
    print(f"\nTop 10 teams by cover percentage:")
    print(df_team_stats.head(10).to_string(index=False))
    
    print(f"\nBottom 10 teams by cover percentage:")
    print(df_team_stats.tail(10).to_string(index=False))
else:
    print("No historical data available")
    df_team_stats = pd.DataFrame()
    all_teams = set()

Completed games with all data: 366

✓ Calculated stats for 30 teams

Top 10 teams by cover percentage:
                team  total_games  covers  non_covers  cover_pct  avg_spread_diff  home_games  away_games
     New York Knicks           24      16           8  66.666667         2.375000          14          10
        Phoenix Suns           25      16           9  64.000000         3.760000          12          13
     Houston Rockets           22      14           8  63.636364         3.181818          10          12
  Los Angeles Lakers           24      15           9  62.500000        -0.520833          11          13
   San Antonio Spurs           25      15          10  60.000000         3.020000          13          12
      Denver Nuggets           24      14          10  58.333333         2.000000          10          14
New Orleans Pelicans           26      15          11  57.692308        -0.961538          15          11
  Philadelphia 76ers           23      13        

## Step 3.5: Calculate Team Coverage with 9-Point Handicap

Calculate each team's spread coverage when given a 9-point handicap. This helps identify teams that consistently beat the spread by significant margins.

In [6]:
# Calculate team spread coverage with 9-point handicap
if len(df_completed) > 0:
    team_stats_handicap_9 = []
    
    for team in all_teams:
        # Games where team was home
        home_games = df_completed[df_completed['home_team'] == team].copy()
        # Games where team was away
        away_games = df_completed[df_completed['away_team'] == team].copy()
        
        # For home games: adjust spread by +9 (making it easier to cover)
        home_covers_handicap = 0
        if len(home_games) > 0:
            adjusted_spread_result = home_games['spread_result_difference'] + 9
            home_covers_handicap = (adjusted_spread_result > 0).sum()
        
        # For away games: adjust spread by -9 (making it easier to cover)
        away_covers_handicap = 0
        if len(away_games) > 0:
            adjusted_spread_result = away_games['spread_result_difference'] - 9
            away_covers_handicap = (adjusted_spread_result < 0).sum()
        
        total_games = len(home_games) + len(away_games)
        total_covers_handicap = home_covers_handicap + away_covers_handicap
        
        if total_games > 0:
            cover_pct_handicap_9 = (total_covers_handicap / total_games) * 100
            
            team_stats_handicap_9.append({
                'team': team,
                'total_games': total_games,
                'covers_handicap_9': total_covers_handicap,
                'cover_pct_handicap_9': cover_pct_handicap_9
            })
    
    # Create DataFrame
    df_team_stats_handicap_9 = pd.DataFrame(team_stats_handicap_9)
    df_team_stats_handicap_9 = df_team_stats_handicap_9.sort_values('cover_pct_handicap_9', ascending=False)
    
    print(f"✓ Calculated 9-point handicap stats for {len(df_team_stats_handicap_9)} teams")
    print(f"\nTop 10 teams by 9-point handicap cover percentage:")
    print(df_team_stats_handicap_9.head(10).to_string(index=False))
else:
    print("No historical data available")
    df_team_stats_handicap_9 = pd.DataFrame()

✓ Calculated 9-point handicap stats for 30 teams

Top 10 teams by 9-point handicap cover percentage:
                 team  total_games  covers_handicap_9  cover_pct_handicap_9
    San Antonio Spurs           25                 24             96.000000
   Philadelphia 76ers           23                 22             95.652174
Oklahoma City Thunder           25                 23             92.000000
    Memphis Grizzlies           24                 22             91.666667
      Detroit Pistons           24                 22             91.666667
           Miami Heat           25                 22             88.000000
        Atlanta Hawks           25                 21             84.000000
       Denver Nuggets           24                 20             83.333333
        Brooklyn Nets           23                 19             82.608696
      Houston Rockets           22                 18             81.818182


## Step 4: Connect Today's Games with Historical Performance

Merge today's games with each team's historical spread coverage statistics (both standard and 9-point handicap).

In [7]:
# Merge today's games with team statistics (including 9-point handicap)
if len(df_today) > 0 and len(df_team_stats) > 0:
    games_with_stats = df_today.copy()
    
    # Merge home team standard stats
    games_with_stats = games_with_stats.merge(
        df_team_stats[['team', 'cover_pct', 'total_games', 'avg_spread_diff']],
        left_on='home_team',
        right_on='team',
        how='left'
    )
    games_with_stats = games_with_stats.rename(columns={
        'cover_pct': 'home_cover_pct',
        'total_games': 'home_total_games',
        'avg_spread_diff': 'home_avg_spread_diff'
    })
    games_with_stats = games_with_stats.drop(columns=['team'])
    
    # Merge home team 9-point handicap stats
    if len(df_team_stats_handicap_9) > 0:
        games_with_stats = games_with_stats.merge(
            df_team_stats_handicap_9[['team', 'cover_pct_handicap_9']],
            left_on='home_team',
            right_on='team',
            how='left'
        )
        games_with_stats = games_with_stats.rename(columns={'cover_pct_handicap_9': 'home_cover_pct_handicap_9'})
        games_with_stats = games_with_stats.drop(columns=['team'])
    
    # Merge away team standard stats
    games_with_stats = games_with_stats.merge(
        df_team_stats[['team', 'cover_pct', 'total_games', 'avg_spread_diff']],
        left_on='away_team',
        right_on='team',
        how='left'
    )
    games_with_stats = games_with_stats.rename(columns={
        'cover_pct': 'away_cover_pct',
        'total_games': 'away_total_games',
        'avg_spread_diff': 'away_avg_spread_diff'
    })
    games_with_stats = games_with_stats.drop(columns=['team'])
    
    # Merge away team 9-point handicap stats
    if len(df_team_stats_handicap_9) > 0:
        games_with_stats = games_with_stats.merge(
            df_team_stats_handicap_9[['team', 'cover_pct_handicap_9']],
            left_on='away_team',
            right_on='team',
            how='left'
        )
        games_with_stats = games_with_stats.rename(columns={'cover_pct_handicap_9': 'away_cover_pct_handicap_9'})
        games_with_stats = games_with_stats.drop(columns=['team'])
    
    # Calculate advantage metrics
    games_with_stats['home_advantage'] = games_with_stats['home_cover_pct'] - games_with_stats['away_cover_pct']
    if 'home_cover_pct_handicap_9' in games_with_stats.columns and 'away_cover_pct_handicap_9' in games_with_stats.columns:
        games_with_stats['home_handicap_advantage'] = games_with_stats['home_cover_pct_handicap_9'] - games_with_stats['away_cover_pct_handicap_9']
    
    print(f"\n✓ Merged today's {len(games_with_stats)} games with team statistics")
    print(f"\nToday's Games with Team Stats:")
    print("="*100)
    
    # Display key columns
    display_cols = ['game_time_est', 'away_team', 'home_team', 'current_spread', 
                    'away_cover_pct', 'home_cover_pct', 'home_advantage']
    if 'home_cover_pct_handicap_9' in games_with_stats.columns:
        display_cols.extend(['away_cover_pct_handicap_9', 'home_cover_pct_handicap_9', 'home_handicap_advantage'])
    print(games_with_stats[display_cols].to_string(index=False))
else:
    print("Cannot merge: Missing today's games or team statistics")
    games_with_stats = pd.DataFrame()


✓ Merged today's 7 games with team statistics

Today's Games with Team Stats:
                game_time_est              away_team             home_team  current_spread  away_cover_pct  home_cover_pct  home_advantage  away_cover_pct_handicap_9  home_cover_pct_handicap_9  home_handicap_advantage
2025-12-12 19:10:00 UTC-05:00          Atlanta Hawks       Detroit Pistons            -7.5       56.000000       54.166667       -1.833333                  84.000000                  91.666667                 7.666667
2025-12-12 19:10:00 UTC-05:00          Chicago Bulls     Charlotte Hornets             2.5       34.782609       50.000000       15.217391                  60.869565                  66.666667                 5.797101
2025-12-12 19:10:00 UTC-05:00    Cleveland Cavaliers    Washington Wizards            14.5       32.000000       27.272727       -4.727273                  60.000000                  63.636364                 3.636364
2025-12-12 19:10:00 UTC-05:00         Indiana Pac

## Step 5: Identify Potential Betting Opportunities

**Focus: Games where home team has better 9-point handicap coverage than away team**

This analysis identifies games where the home team's spread coverage with a 9-point handicap
is higher than the away team's. For each game, we display:
- The matchup and current spread
- Home team's 9-point handicap cover percentage
- Away team's 9-point handicap cover percentage
- The difference between them (home team advantage)

**Note:** This is just one factor to consider. Always do your own research before betting!

In [8]:
# Identify potential betting opportunities
# Focus: Games where home team has better 9-point handicap coverage than away team

if len(games_with_stats) > 0:
    # Filter for games where we have enough data (at least 5 games per team)
    games_with_enough_data = games_with_stats[
        (games_with_stats['home_total_games'] >= 5) & 
        (games_with_stats['away_total_games'] >= 5)
    ].copy()
    
    print(f"Games with sufficient historical data (5+ games per team): {len(games_with_enough_data)}")
    
    # Check if handicap stats are available
    if 'home_cover_pct_handicap_9' in games_with_enough_data.columns and 'away_cover_pct_handicap_9' in games_with_enough_data.columns:
        # Filter for games where home team has better 9-point handicap coverage
        home_handicap_better = games_with_enough_data[
            (games_with_enough_data['home_cover_pct_handicap_9'].notna()) &
            (games_with_enough_data['away_cover_pct_handicap_9'].notna()) &
            (games_with_enough_data['home_cover_pct_handicap_9'] > games_with_enough_data['away_cover_pct_handicap_9'])
        ].copy()
        
        if len(home_handicap_better) > 0:
            # Calculate the difference
            home_handicap_better['handicap_pct_difference'] = (
                home_handicap_better['home_cover_pct_handicap_9'] - 
                home_handicap_better['away_cover_pct_handicap_9']
            )
            
            # Sort by biggest difference (most favorable for home team)
            home_handicap_better = home_handicap_better.sort_values('handicap_pct_difference', ascending=False)
            
            print(f"\n{'='*100}")
            print(f"🏠 HOME TEAM HAS BETTER 9-POINT HANDICAP COVERAGE ({len(home_handicap_better)} games)")
            print(f"{'='*100}")
            print("These games show where the home team's spread coverage with a 9-point handicap")
            print("is better than the away team's. This suggests the home team consistently")
            print("beats spreads by larger margins.")
            print()
            
            # Display each game with detailed information
            for idx, (row_idx, row) in enumerate(home_handicap_better.iterrows(), 1):
                print(f"\nGame {idx}:")
                print(f"  Time: {row['game_time_est']}")
                print(f"  Matchup: {row['away_team']} @ {row['home_team']}")
                print(f"  Current Spread: {row['current_spread']}")
                print(f"  Home Team 9-Point Handicap Cover %: {row['home_cover_pct_handicap_9']:.2f}%")
                print(f"  Away Team 9-Point Handicap Cover %: {row['away_cover_pct_handicap_9']:.2f}%")
                print(f"  Difference: {row['handicap_pct_difference']:.2f}% (Home team advantage)")
                print("  " + "-"*96)
            
            # Also show as a table for easy comparison
            print(f"\n\nSummary Table:")
            print("="*100)
            display_cols = ['game_time_est', 'away_team', 'home_team', 'current_spread',
                            'away_cover_pct_handicap_9', 'home_cover_pct_handicap_9', 'handicap_pct_difference']
            
            # Format the table nicely
            summary_df = home_handicap_better[display_cols].copy()
            summary_df['away_cover_pct_handicap_9'] = summary_df['away_cover_pct_handicap_9'].round(2)
            summary_df['home_cover_pct_handicap_9'] = summary_df['home_cover_pct_handicap_9'].round(2)
            summary_df['handicap_pct_difference'] = summary_df['handicap_pct_difference'].round(2)
            
            # Rename columns for better readability
            summary_df = summary_df.rename(columns={
                'game_time_est': 'Game Time (EST)',
                'away_team': 'Away Team',
                'home_team': 'Home Team',
                'current_spread': 'Spread',
                'away_cover_pct_handicap_9': 'Away 9pt Handicap %',
                'home_cover_pct_handicap_9': 'Home 9pt Handicap %',
                'handicap_pct_difference': 'Difference (%)'
            })
            
            print(summary_df.to_string(index=False))
            
            print(f"\n\nTotal games found: {len(home_handicap_better)}")
            print(f"Average difference: {home_handicap_better['handicap_pct_difference'].mean():.2f}%")
            print(f"Largest difference: {home_handicap_better['handicap_pct_difference'].max():.2f}%")
            print(f"Smallest difference: {home_handicap_better['handicap_pct_difference'].min():.2f}%")
        else:
            print(f"\n{'='*100}")
            print("No games found where home team has better 9-point handicap coverage")
            print(f"{'='*100}")
    else:
        print(f"\n{'='*100}")
        print("9-point handicap stats not available")
        print("Make sure Step 3.5 ran successfully to calculate handicap statistics")
        print(f"{'='*100}")
        
else:
    print("No games data available for analysis")

Games with sufficient historical data (5+ games per team): 7

🏠 HOME TEAM HAS BETTER 9-POINT HANDICAP COVERAGE (5 games)
These games show where the home team's spread coverage with a 9-point handicap
is better than the away team's. This suggests the home team consistently
beats spreads by larger margins.


Game 1:
  Time: 2025-12-12 19:10:00 UTC-05:00
  Matchup: Indiana Pacers @ Philadelphia 76ers
  Current Spread: -6.5
  Home Team 9-Point Handicap Cover %: 95.65%
  Away Team 9-Point Handicap Cover %: 66.67%
  Difference: 28.99% (Home team advantage)
  ------------------------------------------------------------------------------------------------

Game 2:
  Time: 2025-12-12 20:10:00 UTC-05:00
  Matchup: Utah Jazz @ Memphis Grizzlies
  Current Spread: -7.5
  Home Team 9-Point Handicap Cover %: 91.67%
  Away Team 9-Point Handicap Cover %: 69.57%
  Difference: 22.10% (Home team advantage)
  ------------------------------------------------------------------------------------------------

